In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from bs4 import BeautifulSoup
import time
import random

In [2]:
def extractStockData(stock_code):
    
    response = requests.get("https://www.stocklore.ai/api/nse_data.csv?q="+stockCode)
    
    if(response.status_code == 200):
        
        a = response.content.decode("utf-8").split("\n")
        a = [i.split(",") for i in a]
        
        price_df = pd.DataFrame(a[1:-1], columns=a[0])
        price_df.iloc[:,1:] = price_df.iloc[:,1:].astype('float')
        price_df['Date'] = pd.to_datetime(price_df['Date'], format='%Y-%m-%d')
        
        return price_df
    
    else:
        print("Error in Stock Price Data Extraction, Check the stock code name and internet connection")    
        

In [3]:
def getSplitData(stock_code, start_year=2008, end_year=2019, scrape_min_interval=40):
    stocks_split_info = pd.DataFrame(columns = ['NseCode', 'Company', 'OldFV', 'NewFV', 'SplitDate'])
    urls = ["https://www.moneycontrol.com/stocks/marketinfo/splits/homebody.php?sel_year="+str(i) for i in range(start_year, end_year+1)]
    for url in urls:
        time.sleep(scrape_min_interval*(random.random()+1))
        res = requests.get(url)
        soup = BeautifulSoup(res.content,'lxml')
        split_table = soup.find_all('table')[-2]
        split_rows = split_table.find_all('tr')
        rows_with_data = split_rows[3:len(split_rows)-3]
        for row in rows_with_data:
            if(row.find('a')):
                try:
                    code = BeautifulSoup(requests.get("https://www.moneycontrol.com"+row.find('a')['href']).content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
                except:
                    code = 'notScraping'
                company = row.find('a').get_text()
                lst = [row.find_all('td')[i].get_text() for i in range(1,4)]
                stocks_split_info = stocks_split_info.append({'NseCode':code,'Company':company, 'OldFV':lst[0], 'NewFV':lst[1], 'SplitDate':lst[2]}, ignore_index=True)
        
        stocks_split_info['SplitDate'] = pd.to_datetime(stocks_split_info['SplitDate'], format='%d-%m-%Y')        
        stock_split_df = stocks_split_info[stocks_split_info["NseCode"]==stock_code]
    
    return stock_split_df


In [4]:
def getBonusData(stock_code, start_year=2008, end_year=2019, sleep_interval=40):
    bonus_issue_info = pd.DataFrame(columns = ['NseCode', 'Company', 'Ratio', 'AnnouncementDate', 'RecordDate', 'Ex-Bonus'])
    urls = ["https://www.moneycontrol.com/stocks/marketinfo/bonus/homebody.php?sel_year="+str(i) for i in range(start_year, end_year+1)]
    for url in urls:
        time.sleep(sleep_interval*(random.random()+1))
        res = requests.get(url)
        soup = BeautifulSoup(res.content,'lxml')
        bonus_table = soup.find_all('table', class_="b_12 dvdtbl")[0]
        bonus_rows = bonus_table.find_all('tr')
        rows_with_data = bonus_rows[2:]
        for row in rows_with_data:
            if(row.find('a')):
                try:
                    code = BeautifulSoup(requests.get("https://www.moneycontrol.com"+row.find('a')['href']).content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
                except:
                    code = 'notScraping'
                company = row.find('a').get_text()
                lst = [row.find_all('td')[i].get_text() for i in range(1,5)]
                bonus_issue_info = bonus_issue_info.append({'NseCode': code, 'Company':company, 'Ratio':lst[0], 'AnnouncementDate':lst[1], 'RecordDate':lst[2], 'Ex-Bonus':lst[3]}, ignore_index=True)
        
        bonus_issue_info['Ex-Bonus'] = pd.to_datetime(bonus_issue_info['Ex-Bonus'], format='%d-%m-%Y')        
        bonus_issue_df = bonus_issue_info[bonus_issue_info['NseCode']==stock_code]
    
    return bonus_issue_df       

In [ ]:
def adjust_split(stock_code):
    
    price_df = extractStockData(stock_code)
    split_df = getSplitData(stock_code)
    bonus_df = getBonusData(stock_code)
    
    
    

In [61]:
stocks_split_info = pd.DataFrame(columns = ['NseCode', 'Company', 'OldFV', 'NewFV', 'SplitDate'])

In [73]:
a = getBonusData(start_year=2018, end_year=2019)
a

,NseCode,Company,Ratio,AnnouncementDate,RecordDate,Ex-Bonus
0,CAMPALLIED,ORIENTAL AROMAT,1:1,26-10-2018,14-12-2018,13-12-2018
1,,OFS Technologie,6:10,17-09-2018,01-12-2018,29-11-2018
2,,Jiya Eco,1:1,10-10-2018,21-11-2018,19-11-2018
3,,Arfin India,1:5,24-09-2018,14-11-2018,13-11-2018
4,MOTHERSUMI,Motherson Sumi,1:2,07-09-2018,31-10-2018,30-10-2018
5,,Duke Offshore,1:1,31-08-2018,29-10-2018,26-10-2018
6,,CES Limited,27:1,04-09-2018,23-10-2018,22-10-2018
7,,Shashijit Infra,1:5,01-09-2018,15-10-2018,12-10-2018
8,,Suncare Trader,1:1,30-08-2018,12-10-2018,11-10-2018
9,ALPINEHOU,Alpine Hsg,1:3,01-09-2018,12-10-2018,11-10-2018


In [63]:
stocks_split_info['NseCode'][2]

'BRITANNIA'

In [69]:

s = BeautifulSoup(requests.get("https://www.moneycontrol.com/india/stockpricequote/autoancillaries/mothersonsumisystems/MSS01").content, 'lxml').find('div', class_="FL gry10").get_text().split('|')[1][5:].strip()
s

'MOTHERSUMI'